In [1]:
from pycaret.utils import version
version()

1.0.1


In [2]:
from pycaret.datasets import get_data
data = get_data('boston')

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [3]:
from pycaret.regression import *

In [4]:
s = setup(data, target = 'medv', session_id = 123, silent = True)

 
Setup Succesfully Completed.


,Description,Value
0,session_id,123
1,Transform Target,False
2,Transform Target Method,None
3,Original Data,"(506, 14)"
4,Missing Values,False
5,Numeric Features,11
6,Categorical Features,2
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


In [5]:
lr = create_model('lr')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,3.1409,21.4064,4.6267,0.7605,0.4833,0.1363
1,2.3410,8.6772,2.9457,0.8126,0.1396,0.1256
2,3.4407,19.4421,4.4093,0.8059,0.2189,0.1961
3,3.5536,39.5856,6.2917,0.6097,0.2549,0.1851
4,3.0935,20.2982,4.5054,0.8278,0.1743,0.1648
5,3.6330,20.7122,4.5511,0.6147,0.3056,0.2088
6,3.2399,16.8286,4.1023,0.7505,0.2015,0.1613
7,3.9361,39.1510,6.2571,0.6407,0.2615,0.1700
8,2.8611,12.3360,3.5123,0.7794,0.1777,0.1569
9,3.8989,28.4996,5.3385,0.6702,0.2446,0.1631


In [6]:
dt = create_model('dt')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,4.4194,34.1508,5.8439,0.6179,0.2283,0.1595
1,2.2389,10.5872,3.2538,0.7714,0.1353,0.1073
2,2.3556,9.8211,3.1339,0.9019,0.1911,0.1473
3,2.4694,11.1575,3.3403,0.8900,0.1873,0.1475
4,2.4229,10.8417,3.2927,0.9080,0.1607,0.1317
5,2.2686,10.1189,3.1810,0.8118,0.1570,0.1237
6,2.0771,6.7054,2.5895,0.9006,0.1190,0.1000
7,3.5286,33.0209,5.7464,0.6970,0.2068,0.1544
8,4.1000,53.2100,7.2945,0.0483,0.2943,0.2465
9,3.3829,18.4691,4.2976,0.7863,0.2111,0.1545


In [7]:
rf = create_model('rf')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,2.9202,15.2739,3.9082,0.8291,0.1501,0.1116
1,1.7927,5.7213,2.3919,0.8765,0.1096,0.0932
2,2.0494,6.2177,2.4935,0.9379,0.1556,0.1320
3,2.7498,16.1704,4.0212,0.8405,0.1837,0.1498
4,2.1685,7.5993,2.7567,0.9355,0.1190,0.1022
5,1.9155,6.8230,2.6121,0.8731,0.1430,0.1083
6,1.4799,3.6564,1.9122,0.9458,0.0853,0.0678
7,2.7569,20.9588,4.5781,0.8077,0.1742,0.1263
8,2.5172,14.6935,3.8332,0.7372,0.2011,0.1542
9,1.9336,8.3841,2.8955,0.9030,0.1318,0.0864


In [8]:
lightgbm = create_model('lightgbm')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,2.8951,14.9672,3.8688,0.8325,0.1341,0.1080
1,1.6646,4.6609,2.1589,0.8994,0.1149,0.0885
2,2.2978,8.2029,2.8641,0.9181,0.1635,0.1415
3,2.2764,16.7234,4.0894,0.8351,0.1969,0.1386
4,1.8088,5.2475,2.2907,0.9555,0.1308,0.0988
5,2.2089,8.3934,2.8971,0.8439,0.1378,0.1118
6,2.0330,6.2837,2.5067,0.9069,0.1115,0.0924
7,3.0730,26.0023,5.0992,0.7614,0.1787,0.1291
8,2.4877,13.0345,3.6103,0.7669,0.1854,0.1484
9,2.3070,10.2140,3.1959,0.8818,0.1211,0.0944


In [14]:
xgboost = create_model('xgboost')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,2.6697,13.2210,3.6361,0.8521,0.1411,0.1058
1,1.6829,4.8483,2.2019,0.8953,0.1125,0.0912
2,2.0862,7.1808,2.6797,0.9283,0.1614,0.1251
3,2.9884,25.2462,5.0246,0.7510,0.1865,0.1399
4,1.9248,6.2716,2.5043,0.9468,0.1202,0.0977
5,2.1730,8.8725,2.9787,0.8349,0.1515,0.1185
6,1.9831,6.2714,2.5043,0.9070,0.1113,0.0887
7,2.6319,24.4130,4.9410,0.7760,0.1655,0.1077
8,2.0116,8.9471,2.9912,0.8400,0.1598,0.1204
9,2.4887,11.8052,3.4359,0.8634,0.1452,0.1035


In [10]:
stacker = stack_models(estimator_list = [lr,dt,xgboost,lightgbm], meta_model=rf, improve_only = False)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,3.0521,16.7229,4.0894,0.8129,0.1575,0.1160
1,1.6468,5.2055,2.2816,0.8876,0.1101,0.0879
2,2.1672,7.1519,2.6743,0.9286,0.1671,0.1372
3,2.8635,19.2893,4.3920,0.8098,0.1804,0.1441
4,2.0389,6.7317,2.5945,0.9429,0.1225,0.0974
5,2.0597,8.8073,2.9677,0.8362,0.1544,0.1137
6,1.9456,6.3369,2.5173,0.9061,0.1114,0.0886
7,2.7678,27.6261,5.2561,0.7465,0.1792,0.1155
8,2.4698,18.0962,4.2540,0.6763,0.2048,0.1496
9,2.3043,12.1445,3.4849,0.8595,0.1432,0.0964


In [11]:
stacker

[LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False),
 DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=123, splitter='best'),
 XGBRegressor(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=-1, num_parallel_tree=1,
              objective='reg:squarederror', random_state=123, reg_alpha=0,
              reg_lam

In [12]:
predict_model(stacker);

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Stacking Regressor,2.3345,14.8453,3.853,0.8163,0.1653,0.1268
